In [1]:
#import libraries
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
import re 
import openpyxl
import time
#from requests.exceptions import HTTPError

In [2]:
def replace_nan(ar):
    for i in range(len(ar)):
        if ar[i]==None:
            ar[i]=0
    return ar

def del_substr(x):
    for u in range(len(x)):
        r=re.sub('<highlighttext>','',str(x[u]))
        x[u]=re.sub('</highlighttext>','',r)
    return x

In [3]:
#request parameters
s=requests.Session()
url = 'https://api.hh.ru/vacancies'
params={'text':('data scientist')}

In [4]:
#get list of all previous vacancies
url_list=list()
try:
    file=pd.read_excel('data scientist vacancies.xlsx')
    url_list=list(file['URL'])
except (FileNotFoundError, OSError):
    print('File has not been found')

In [5]:
try:
    wb = openpyxl.load_workbook('data scientist vacancies.xlsx',data_only=True)
    sheet = wb.active
except (FileNotFoundError, OSError):
    print('File has not been found')

In [7]:
try:
    while True:
        #get response
        page=s.get(url, params=params,stream=True)
        if page.status_code == 200:
            result=page.json()['items']

            #get salary, anme and URL
            salary=[s.get('salary') for s in result]
            for i in range(len(salary)):
                if salary[i]!=None:
                    salary[i]=salary[i].get('from')
            salary=replace_nan(salary)
            URL=[u.get('url') for u in result]
            vacancy_name=[n.get('name') for n in result]

            #get responsibilities and requirements
            respons=[s.get('snippet') for s in result]
            requirement=[r.get('requirement') for r in respons]
            responsibility=[q.get('responsibility') for q in respons]
            responsibility=replace_nan(responsibility)
            requirement=replace_nan(requirement)
            requirement=del_substr(requirement)
            responsibility=del_substr(responsibility)

            #create dataframe
            data=pd.concat([pd.Series(URL),pd.Series(vacancy_name), pd.Series(salary), pd.Series(requirement), pd.Series(responsibility)],axis=1)
            data.columns=['URL','Name','Salary','Requirements','Responsibilities']
            data=data.reset_index(drop=True)

            #append to excel file, given vacancies have not been added yet
            for i in range(len(data.iloc[:,0])):
                if data.iloc[i,0] not in url_list: 
                    sheet.append(list(data.iloc[i,:]))
                    url_list.append(data.iloc[i,0])

            time.sleep(30)
        else:
            print('Connection error has occured: {}'.format(page.status_code))
except KeyboardInterrupt:
    pass

In [8]:
try:
    wb.save('data scientist vacancies.xlsx')
    print('Workbook saved')
except PermissionError:
    print ('Close the workbook and try again')

Workbook saved
